In [1]:
import os
import pandas as pd
import json

from dict_process import *

sheets = ['传送模设计科', '设计二科', '设计一科', '研磨组', '车铣钻组', 'CNC组', '线割组']

In [2]:
with open('dict/touse.json', 'r') as f:
    errors = json.load(f)

path = '/home/keae/Mould/data'
filelist = os.listdir(path)
filelist.sort()

print('识别错误')
for file in filelist[:]:
    print(os.path.join(path, file))
    dfs = pd.read_excel(os.path.join(path, file), header=1, sheet_name=None)

    writer = pd.ExcelWriter('results/'+file+'.xls')
    for sheet in sheets[:]:
        df = dfs[sheet]
        df = df[['异常单编码', '品管异常描述']]
        df = df.replace('\n', '', regex=True)
        df[['错误位置', '错误类型', '原文错误']] = pd.DataFrame([['', '', '']], index=df.index)

        dfs[sheet] = recog(df, errors[sheet])
        dfs[sheet].to_excel(writer, sheet)

    writer.save()


识别错误
/home/keae/Mould/data/2017年（9.18-9.24）品质异常一览表（修改）(1).xls


/home/keae/Mould/.venv/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3361: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  if (await self.run_code(code, result,  async_=asy)):


/home/keae/Mould/data/2017年（9.25-10.8）品质异常一览表（修改）(1).xls
/home/keae/Mould/data/改表2017年（9-01-9..10）.xls
/home/keae/Mould/data/改表2017年（9.11-9.17）.xls


In [2]:
path = '/home/keae/Mould/data'
filelist = os.listdir(path)
filelist.sort()

errors = {}
print('生成字典：')
for file in filelist[:]:
    print(os.path.join(path, file), end=': ')

    df = pd.read_excel(os.path.join(path, file), header=1, sheet_name=None)
    for sheet in sheets[0:]:
        print(sheet, end=',')
        tmp = df[sheet]

        columns = tmp.columns.values.tolist()
        columns = [c.replace('\n', '').replace(' ', '') for c in columns]
        tmp.columns = columns

        tmp = tmp[['异常单编码', '品管异常描述', '错误位置1', '错误位置2', '原错误类型', '归纳错误类型']]
        tmp = tmp.replace('\n', '', regex=True)
        tmp = tmp.replace(' ', '', regex=True)

        if sheet in errors:
            error = errors[sheet]
        else:
            error = {'错误位置': [], '错误类型': {}}

        errors[sheet] = gener(tmp, error)

    print()


with open('dict/extract.json', 'w') as f:
    json.dump(errors, f, sort_keys=True, indent=4, ensure_ascii=False)


生成字典：
/home/keae/Mould/data/2017年（9.18-9.24）品质异常一览表（修改）(1).xls: 传送模设计科,设计二科,设计一科,研磨组,车铣钻组,CNC组,线割组,
/home/keae/Mould/data/2017年（9.25-10.8）品质异常一览表（修改）(1).xls: 传送模设计科,设计二科,设计一科,研磨组,车铣钻组,CNC组,线割组,
/home/keae/Mould/data/改表2017年（9-01-9..10）.xls: 传送模设计科,设计二科,设计一科,研磨组,车铣钻组,存在空值,存在空值,存在空值,CNC组,线割组,
/home/keae/Mould/data/改表2017年（9.11-9.17）.xls: 传送模设计科,设计二科,设计一科,研磨组,存在空值,车铣钻组,CNC组,存在空值,线割组,


In [3]:
with open('dict/extract.json', 'r') as f:
    extract = json.load(f)
with open('dict/expert.json', 'r') as f:
    expert = json.load(f)

d = {}
for sheet in sheets[:]:
    d[sheet] = merge(extract[sheet], expert[sheet])


with open('dict/merge.json', 'w') as f:
    json.dump(d, f, sort_keys=True, indent=4, ensure_ascii=False)


In [3]:
with open('dict/merge.json', 'r') as f:
    merge = json.load(f)

for sheet in sheets[:]:
    merge[sheet]['错误类型'] = rever(merge[sheet]['错误类型'])


with open('dict/touse.json', 'w') as f:
    json.dump(merge, f, sort_keys=True, indent=4, ensure_ascii=False)
